# QAOA - Quantum Approximate Optimization Algorithm

In this notebook I would like to cover one of the most prominent algorithms in the quantum computing in the last few years.

In this tutorial you will learn the following:
- How to use pyQuil - python platform for quantum computing?
- What is QAOA?
TODO

## Prerequisites

Before we can go into QAOA itself, you should prepare a couple of things:
1. Configure your Forest API Key - go to http://forest.rigetti.com, find section "Get API Key" and follow the instructions
2. Install [pyQuil](https://github.com/rigetticomputing/pyquil) and [Grove](https://github.com/rigetticomputing/grove)
3. Learn the basics of quantum computing by yourself - there are plenty of great resources about the basics in the web and I decided I don't want to reinvent the wheel in this tutorial. I recommend going to [pyQuil documentation](http://pyquil.readthedocs.io/en/stable/).

Even though I tried to make this tutorial as accessible to the not-quantum-computing people as possible, you probably need some level of familiarity with the following concepts:
- qubits
- quantum gates
- superposition
- entanglement

You don't need to go very in-depth - I think spending about 30 minutes with the pyQuil docs should be enough for the beginning. And if something is still confusing later onyou can always come back to the documentation when you need it :)


## QAOA - intro

QAOA is an algorithm for solving a broad range of optimization problems using NISQ (Noisy Intermediate-Scale Quantum) devices. 

Let's start easy and instead of going straight to the TSP we will solve MaxCut problem. This part is based on the [Rigetti's tutorial on QAOA](https://grove-docs.readthedocs.io/en/latest/qaoa.html).

### MaxCut - explanation

In the MaxCut problem we start with a graph and we want to divide it into two subgraphs in such way, that the edges between them would have the higher possible sum.

#TODO Picture and finish this section

### QAOA solves MaxCut - quick take

We will use a class maxcut_solver which we can find in the grove library.

Now, let's take a graph from the previous section.
We encode it as a list of tuples, where each tuple represents an edge between given nodes.

In [2]:
first_graph = [(0, 1), (1, 2), (2, 3), (3, 0)]

TODO: picture and use a different graph with single solution

Now let's try to solve MaxCut problem for this graph using maxcut_solver from the grove library.

Keep in mind that runtime of this algorithm are non-deterministic - it should run in under a minute, but sometimes it takes longer.
TODO: make it red.

In [1]:
import numpy as np
from grove.pyqaoa.maxcut_qaoa import maxcut_qaoa
import pyquil.api as api
qvm_connection = api.QVMConnection()

In [19]:
%%capture 
#%%capture supresses printing. 
#get_angles() prints out a lot of stuff that we don't care about right now.
maxcut_solver = maxcut_qaoa(graph=first_graph)
betas, gammas = maxcut_solver.get_angles()

Now that we have the values of betas and gammas we can run the actual program.

In [21]:
angles = np.hstack((betas, gammas))
param_prog = maxcut_solver.get_parameterized_program()
prog = param_prog(angles)
qubits = [0, 1, 2, 3]
measurements = qvm_connection.run_and_measure(prog, qubits, trials=1000)

And see what are the results:

In [29]:
from collections import Counter
measurements = [tuple(measurement) for measurement in measurements]
measurements_counter = Counter(measurements)
measurements_counter.most_common()

[((0, 1, 0, 1), 278),
 ((1, 0, 1, 0), 253),
 ((0, 1, 1, 0), 89),
 ((0, 0, 1, 1), 78),
 ((1, 0, 0, 1), 74),
 ((1, 1, 0, 0), 72),
 ((0, 1, 1, 1), 29),
 ((1, 0, 1, 1), 18),
 ((1, 1, 0, 1), 17),
 ((0, 0, 0, 1), 16),
 ((0, 0, 0, 0), 15),
 ((0, 1, 0, 0), 13),
 ((1, 1, 1, 0), 13),
 ((1, 0, 0, 0), 12),
 ((0, 0, 1, 0), 12),
 ((1, 1, 1, 1), 11)]

So as we can see, the correct results are at the top of our lists - so everything seems good.

But why?

We will go through the code step by step in the next section

### QAOA solves MaxCut - details

#### Finding the right angles

In [30]:
# We initialize the maxcut_qaoa object with our graph
maxcut_solver = maxcut_qaoa(graph=first_graph)
# The QAOA algorithm tries to find the optimal values of betas and gammas.
# This line is where all the optimization takes place.
betas, gammas = maxcut_solver.get_angles()
print("Values of betas:", betas)
print("Values of gammas:", gammas)

                     models will be ineffective
	Parameters: [3.06785487 1.05495912] 
	E => -1.7505388790874525
	Parameters: [3.38890945 0.99362428] 
	E => -2.764202773444961
	Parameters: [3.46025492 1.06722608] 
	E => -2.6428589801781284
	Parameters: [3.60294584 1.01815822] 
	E => -2.8601048306171584
	Parameters: [3.60294584 1.01815822] 
	E => -2.8546884930214387
	Parameters: [3.60294584 0.96909035] 
	E => -2.816730274478843
	Parameters: [3.53160038 1.00589125] 
	E => -2.9042787706429687
	Parameters: [3.49592765 0.92615597] 
	E => -2.949346734906352
	Parameters: [3.49592765 0.92615597] 
	E => -2.930042189056577
	Parameters: [3.5182231  0.82878692] 
	E => -2.9941800668645073
	Parameters: [3.5182231  0.82878692] 
	E => -2.951103255086815
	Parameters: [3.5182231  0.82878692] 
	E => -2.957154664312718
	Parameters: [3.5182231  0.82878692] 
	E => -2.9938695502656736
	Parameters: [3.5182231  0.82878692] 
	E => -2.993420645972215
	Parameters: [3.54219072 0.80262378] 
	E => -2.998907800917262


This time we have not supressed the output and with every iteration of the algorithm you can see two lines being printed.
<br>
The first one is parameters - these are values of betas and gammas.
<br>
The second one is the energy value - it's something we try to minize. As you can probably see, the minimization process isn't ideally smooth. Value is generally getting smaller and smaller, but not necessarily every step yields an improvement.
<br>
We will deal with what exactly this energy means later on.

#### Creating a circuit

In [32]:
# We create an array of angles with correct format
angles = np.hstack((betas, gammas))
print(angles)

[3.5333237  0.78691328]


In [38]:
# We take a template for quil program from the maxcut_solver.
param_prog = maxcut_solver.get_parameterized_program()
# We initialize this program with the angles we have found
prog = param_prog(angles)
# Now we can print the program. 
# Some of the values you see here are the angles we calculated earlier.
print(prog)
print("Number of gates:", len(prog))

H 0
H 1
H 2
H 3
CNOT 0 1
RZ(0.7869132823909201) 1
CNOT 0 1
X 0
PHASE(0.39345664119546003) 0
X 0
PHASE(0.39345664119546003) 0
CNOT 0 3
RZ(0.7869132823909201) 3
CNOT 0 3
X 0
PHASE(0.39345664119546003) 0
X 0
PHASE(0.39345664119546003) 0
CNOT 1 2
RZ(0.7869132823909201) 2
CNOT 1 2
X 0
PHASE(0.39345664119546003) 0
X 0
PHASE(0.39345664119546003) 0
CNOT 2 3
RZ(0.7869132823909201) 3
CNOT 2 3
X 0
PHASE(0.39345664119546003) 0
X 0
PHASE(0.39345664119546003) 0
H 0
RZ(-7.066647405913116) 0
H 0
H 1
RZ(-7.066647405913116) 1
H 1
H 2
RZ(-7.066647405913116) 2
H 2
H 3
RZ(-7.066647405913116) 3
H 3

Number of gates: 44


#### Running the quantum program

In [41]:
# These are just the ids of qubits we want to use.
# It's not very important if you don't use the real QPU.
qubits = [0, 1, 2, 3]
# Here we connect to the Forest API and run our program there.
# We do that 1000 times and after each one we measure the output.
measurements = qvm_connection.run_and_measure(prog, qubits, trials=1000)

#### Analyzing the results

In [42]:
# Since list of 1000 elements is hard to analyze, we use Counter
from collections import Counter
# This is just a hack - we can't use Counter on a list of lists but we can on a list of tuples.
measurements = [tuple(measurement) for measurement in measurements]
measurements_counter = Counter(measurements)
# This line gives us the results in the diminishing order
measurements_counter.most_common()

[((1, 0, 1, 0), 272),
 ((0, 1, 0, 1), 267),
 ((1, 1, 0, 0), 92),
 ((0, 0, 1, 1), 73),
 ((1, 0, 0, 1), 69),
 ((0, 1, 1, 0), 68),
 ((0, 1, 1, 1), 21),
 ((1, 1, 0, 1), 20),
 ((0, 0, 0, 1), 18),
 ((1, 1, 1, 0), 17),
 ((1, 0, 1, 1), 17),
 ((0, 0, 1, 0), 15),
 ((1, 1, 1, 1), 15),
 ((0, 0, 0, 0), 14),
 ((1, 0, 0, 0), 11),
 ((0, 1, 0, 0), 11)]

#### Conclusions

This is a good time to cover some fundamental topics:
<br>
**1. Probabilistic nature of quantum computing**
<br>
One of the most basic rules of the quantum mechanics is that the act of observation changes the state of the system that we observe. 
<br>
Even though our algorithm produces some state which is a combination of all the states you see above, we can't measure it directly. What we can do (and we did) is to measure it repeatedly and get some distribution of the results.
<br>
Fortunately, we use a simulator and we can look how the state we produced looks exactly:


In [46]:
wf = qvm_connection.wavefunction(prog)
print(wf)

(0.0003791911-0.1248956553j)|0000> + (-0.1246226478+0.0005903969j)|0001> + (-0.1246226478+0.0005903969j)|0010> + (-0.2503783672+0.1251031969j)|0011> + (-0.1246226478+0.0005903969j)|0100> + (-0.4996162177-0.126410765j)|0101> + (-0.2503783672+0.1251031969j)|0110> + (-0.1246226478+0.0005903969j)|0111> + (-0.1246226478+0.0005903969j)|1000> + (-0.2503783672+0.1251031969j)|1001> + (-0.4996162177-0.126410765j)|1010> + (-0.1246226478+0.0005903969j)|1011> + (-0.2503783672+0.1251031969j)|1100> + (-0.1246226478+0.0005903969j)|1101> + (-0.1246226478+0.0005903969j)|1110> + (0.0003791911-0.1248956553j)|1111>


This is hard to read and interpret, so let's print it in a different form.

In [52]:
print("Probability amplitudes for all the possible states:")
for state_index in range(inst.nstates):
    print(inst.states[state_index], wf[state_index])

Probability amplitudes for all the possible states:
0000 (0.0003791910996818887-0.12489565526984793j)
0001 (-0.12462264778937672+0.0005903969201245737j)
0010 (-0.1246226477893766+0.0005903969201245668j)
0011 (-0.25037836723769236+0.1251031969382479j)
0100 (-0.12462264778937665+0.0005903969201245113j)
0101 (-0.49961621774324116-0.12641076498844267j)
0110 (-0.2503783672376924+0.1251031969382479j)
0111 (-0.12462264778937661+0.0005903969201245113j)
1000 (-0.12462264778937661+0.0005903969201245113j)
1001 (-0.2503783672376924+0.1251031969382479j)
1010 (-0.49961621774324116-0.12641076498844267j)
1011 (-0.12462264778937665+0.0005903969201245113j)
1100 (-0.25037836723769236+0.1251031969382479j)
1101 (-0.1246226477893766+0.0005903969201245668j)
1110 (-0.12462264778937672+0.0005903969201245737j)
1111 (0.0003791910996818887-0.12489565526984793j)


These numbers are still not very easy to interpret, but we can calculate the actual probabilities based on this:

In [53]:
print("Probabilities of measuring given states:")
for state_index in range(inst.nstates):
    print(inst.states[state_index], np.real(np.conj(wf[state_index])*wf[state_index]))


Probabilities of measuring given states:
0000 0.015599068491174772
0001 0.015531152910558335
0010 0.015531152910558305
0011 0.07834013666478276
0100 0.015531152910558317
0101 0.26559604653702507
0110 0.07834013666478279
0111 0.015531152910558307
1000 0.015531152910558307
1001 0.07834013666478279
1010 0.26559604653702507
1011 0.015531152910558317
1100 0.07834013666478276
1101 0.015531152910558305
1110 0.015531152910558335
1111 0.015599068491174772


These results 

TODO: create a nice histogram

## QAOA - how does it work?

## Additional resources

- https://arxiv.org/pdf/1801.00862.pdf